# The Relationship between Turnout and Topics

In [24]:
import stata_setup

stata_setup.config('/Applications/StataNow', 'mp')

In [25]:
%%stata

clear all
set more off
set varabbrev off
version 18

* Paths
global PROCESSED_DATA "processed_data"
global TABLES "tables"

In [26]:
%%stata

import delimited using "$PROCESSED_DATA/proposals_panel.csv", clear

* Parse date
gen day = date(date, "YMD")
format day %td
gen month = month(day)
gen quarter = quarter(day)

gen year = year(day)
encode gecko_id, gen(token)
encode space,    gen(dao)

* Dependent variables
local dep non_whale_participation whale_participation

* Independent variables
local topic protocol_security treasury_expenditure user_incentive_increase  tokenomics voting_mechanism_change

* Label variables
label var user_incentive_increase "\${\it User Incentive Increase}_{i,t}\$"
label var treasury_expenditure "\${\it Treasury Expenditure}_{i,t}\$"
label var voting_mechanism_change "\${\it Voting Mechanism Change}_{i,t}\$"
label var protocol_security "\${\it Protocol Security}_{i,t}\$"
label var tokenomics "\${\it Tokenomics}_{i,t}\$"

## Proposal Characteristics

In [27]:
%%stata

******************************************************
* LOGISTIC REGRESSIONS
******************************************************

eststo clear

foreach d of local dep {
    * Run regression
    reghdfe `d' `topic', absorb(year dao) vce(cluster token)
    eststo `d'

    * Tag FE rows (Y/N) for this spec
    estadd local fe_token "Y"
    estadd local fe_time  "Y"
}

* Export LaTeX table
esttab                                                      ///
    non_whale_participation whale_participation                         ///
    using "$TABLES/reg_turnout_topic.tex", replace          ///
    se star(* 0.10 ** 0.05 *** 0.01) nogaps nocon           ///
    b(%9.3f) se(%9.2f)                                      ///
    label nogaps booktabs nonotes noomitted eqlabels(none)  ///
    mtitles("\${\it Turnout}^{Small}_{i,t}\$"               ///
    "\${\it Turnout}^{Whale}_{i,t}\$")                      ///  
    substitute("\_" "_")                                    ///
    stats(fe_token fe_time N r2_a,                          ///
        fmt(0 0 %9.0fc %9.3f)                                   ///
         labels("Token FE" "Year FE" "Observations" "Adjusted R²"))